<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>

# How to train YOLO v7 using the Ikomia API

Object detection is an important task in Computer Vision. Lots of companies use this technology:


*   **Smart city** : crowd analysis or infrastructure inspection
*   **Retail** : inventory tracking or customer tracking
*   **Autonomous vehicles** : vehicle or pedestrian detection
*   **Security** : facial recognition or potential threats detection
*   **Manufacturing** : quality control or defect detection
*   **Healthcare** : many disease recognition


### Overview of the YOLOv7 model

| Release date | July 2022        |
|--------------|------------------|
| Model type   | Object detection |
| Architecture | YOLO, CNN        |
| Framework    | PyTorch          |
| Github stars | >10k             |

Compared to its predecessors, YOLOv7 introduces several architectural reforms that contribute to improved performance. These include:
- Architectural reform:
  - Model scaling for concatenation-based models
  - E-ELAN (Extended Efficient Layer Aggregation Network)
- Trainable Bag-of-Freebies (BoF)
  - Planned re-parameterized convolution.
  - Coarse for auxiliary and fine for lead loss  

These enhancements result in a 13.7% higher Average Precision (AP) on the COCO dataset compared to YOLOv6.


- Paper : [YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors](https://arxiv.org/abs/2207.02696)

- Code : [GitHub repository](https://github.com/wongkinyiu/yolov7)

If you like this tutorial, you can support our project here [Ikomia API GitHub](https://github.com/Ikomia-dev/IkomiaApi).

ENJOY 🥰 !!

<p float="left">
  <img src="https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/img_aerial.jpg?raw=true" width="400" /> 
  <img src="https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/img_aerial_bbox.png?raw=true" width="400" />
</p>

## Setup

Please use a GPU for this tutorial.

In the menu, select "Runtime" then "Change runtime type", choose GPU in "Hardware accelerator".

Check your GPU with the following command:

In [ ]:
!nvidia-smi

First of all, you need to install Ikomia API pip package.

In [ ]:
!pip install ikomia numpy==1.23.5



---


**-Google Colab ONLY- Restart runtime**

Click on the "RESTART RUNTIME" button at the end the previous window.





---

Ikomia API has already more than 280 pre-integrated algorithms but the most interesting algorithms are in [Ikomia HUB](https://github.com/Ikomia-hub). 

We push regularly state-of-the-art algorithms from individual repos (think of YOLO v7 for example) or from companies (Facebook Detectron2 or Ultralytics/YOLOv5 for example).

## How to train YOLO v7 on a custom YOLO dataset

Download your dataset from your preferred tool. In this example, we use a dataset from **Roboflow** which is a great annotation platform used by many developers and companies. The dataset is exported in YOLO format.

In [ ]:
!curl -L "https://universe.roboflow.com/ds/W8grf6DmCF?key=QkQVA4pg66" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!cp $(pwd)/valid/* $(pwd)/train/

In order to train YOLOv7 on your custom dataset, please create a new workflow from scratch.

Then you need 2 components:

1.   A YOLO dataset loader which loads dataset in YOLO format and convert it to an Ikomia format
2.   The YOLOv7 training algorithm which loads dataset in Ikomia format

Add these 2 previous algorithms to your workflow and then it will automagically download all algorithms from Ikomia Hub and install all the Python dependencies (the 1st time, it can take a while, be patient ! ).

** -Google Colab ONLY- ** 
If you want to monitor your training results, you can use TensorBoard by running the following cell.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /root/Ikomia/Tensorboard

Now, it's time to train your model !

In [ ]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils import ik
import os

#----------------------------- Step 1 -----------------------------------#
# Create a workflow which will take your dataset as input and
# train a YOLOv7 model on it
#------------------------------------------------------------------------#
wf = Workflow()

#----------------------------- Step 2 -----------------------------------#
# First you need to convert darknet YOLO format to IKOMIA format.
# Add an Ikomia dataset converter to your workflow.
#------------------------------------------------------------------------#
dataset_yolo = wf.add_task(ik.dataset_yolo(dataset_folder=os.getcwd()+"/train", class_file=os.getcwd()+"/train/_darknet.labels"))

#----------------------------- Step 3 -----------------------------------#
# Then, you want to train a YOLOv7 model.
# Add YOLOv7 training algorithm to your workflow
#------------------------------------------------------------------------#
yolo7 = wf.add_task(ik.train_yolo_v7(output_folder=os.getcwd(), batch_size="16", epochs="10"), auto_connect=True)# <-- Reduce the batch size if you encounter some "Cuda out of memory"

#----------------------------- Step 4 -----------------------------------#
# Execute your workflow.
# It automatically runs all your tasks sequentially.
#------------------------------------------------------------------------#
wf.run()

## How to execute YOLOv7 on images

Once the training is finished, you may want to experiment the fresh model on new test images. Just use the following code to create a YOLO v7 inference workflow.
Then run and test !

In [ ]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils import ik

# Create your workflow for YOLO inference
wf = Workflow()

# Add YOLO v7 to your workflow
yolov7 = wf.add_task(ik.infer_yolo_v7(model_weight_file=os.getcwd()+"/[timestamp]/weights/best.pt", conf_thres="0.4"), auto_connect=True) # <-- replace "[timestamp]" by the correct folder name (/content/ for Google Colab)

## Run and display your results

In [ ]:
from ikomia.utils.displayIO import display
from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Apply YOLO v7 on your image
# By default, YOLOv7 runs with a pre-trained model based on COCO
# To use your custom model, set the parameters in the previous cell
wf.run_on(path=os.getcwd()+"/test/airport_246_jpg.rf.3d892810357f48026932d5412fa81574.jpg")

# Get YOLO v7 image result
img_bbox = yolov7.get_image_with_graphics()

# Display in Colab
display(img_bbox)

## -Google Colab ONLY- Save your YOLO v7 model in your Google Drive account



In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Uncomment and insert the correct "folder name" in the following path
#%cp /content/folder_name/weights/best.pt /content/gdrive/MyDrive

## -Google Colab ONLY- Download directly your custom model

In [ ]:
#from google.colab import files

# Uncomment and insert the correct "folder name" in the following path
#files.download('/content/folder_name/weights/best.pt')